# Wykrywanie społeczności w sieciach

**Dominika Szydło** i **Oliwier Kaszyca**

---

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import pandas as pd
from pathlib import Path
from gensim.models import Word2Vec
from IPython.display import Code, display

from src.utils import generate_gcn_summary

In [2]:
DATA_PATH = Path("data")
EMBBEDDINGS_PATH = Path("embbeddings")
RESULTS_PATH = Path("results")

---

# Opis problemu

## Typ problemu

Rozważany problem to klasyfikacja wierzchołków w grafie reprezentującym sieć społecznościową. 


## Zbiór danych


Badania przeprowadzono na zmodyfikowanej sieci *email-Eu-core* składającym się z 986 wierzchołków, przyporządkowanych do 37 grup. Z sieci została usunięta kierunkowość, w celu otrzymania prostej relacji między pracownikami. Wizualizację badanej sieci przedstawiono poniżej. 

<center><img src="images/network.png" width="600" height="300"></center>

Badany zbiór danych charakteryzuje się wysokim stopniem niezbalansowania, co wpłynęło na sposób przygotowania eksperymentów.

<center><img src="images/departments_cleaned.png"A width="600" height="300"><center>

### Dane wejściowe

Danymi wejściowymi będzie informacja o pracowniku, reprezentowana jako wektor wygenerowany przez metodę [`node2vec`](https://github.com/aditya-grover/node2vec) ($X$) oraz informacja o przynależności do działu ($Y$).

In [9]:
pd.read_csv(DATA_PATH / "train_data_64.tsv", sep=" ")

,user_id,department_id,embbedding_id
0,0,1,274
1,1,1,214
2,2,19,62
3,3,19,121
4,4,19,63
...,...,...,...
971,1000,4,746
972,1001,19,670
973,1002,1,908
974,1003,6,912


In [8]:
model = Word2Vec.load(str(EMBBEDDINGS_PATH / "graph_64.model"))
model.wv[0]

array([ 0.2546108 , -0.23057798,  0.21836537,  0.46526358, -0.17184058,
        0.08660654, -0.1691932 ,  0.22447565, -0.04186373, -0.10044232,
        0.09406602, -0.14495793, -0.18923284,  0.12602815,  0.02033304,
        0.13001777, -0.03674696, -0.15397125, -0.0108666 ,  0.1237722 ,
        0.20768197,  0.02528708,  0.06163227,  0.05012866, -0.0123972 ,
       -0.10703237, -0.22338434, -0.17893176, -0.04474849,  0.24680692,
       -0.23790954,  0.17568135, -0.18129091, -0.04467465,  0.12230913,
       -0.10467809,  0.14315628,  0.06247655,  0.01302138,  0.35156024,
       -0.3315806 , -0.28147262,  0.19358933, -0.3252379 , -0.16074616,
       -0.29917642,  0.16210511,  0.01038771, -0.09245422,  0.21811223,
       -0.08821449,  0.02596824, -0.05547078,  0.09915774, -0.07835271,
        0.06661233, -0.02587465,  0.13907675, -0.1931055 ,  0.04680524,
       -0.0336911 ,  0.15107258, -0.00409859,  0.22574055], dtype=float32)

Ważnym krokiem w tworzeniu wektorów osadzeń było przygotowanie grafu, który reprezentuje sieć w badanym zbiorze. Wierzchołki w stworzonym grafie, w wersji podstawowej zawierały informację o przynależności do danego działu. Aby uniknąć niekontrolowanego wycieku informacji, ta cecha została usunięta tuż przed wywołaniem metody generującej wektory osadzeń. Ostatecznie graf posiadał tylko dane na temat połączeń między użytkownikami, a cecha mówiąca o dziale została przypisana do niezależnego zbioru $Y$. 

### Dane wyjściowe

Jako dane wyjściowe otrzymujemy identyfikator klasy $c$, który określa przynależność pracownika do działu (klasy).

## Hipoteza badawcza

TODO

# Opis wybranych modeli

## Graph Attention Network (GAT)

TODO

## Graph Convolutional Network (GCN) 

Grafowe sieci neuronowe znane są od lat, jednak dopiero rozwój i popularyzacja uczenia głębokiego pozwoliła na efektywne implementacje. Najpopularniejszym obecnie modelem grafowej sieci neuronowej jest **grafowa konwolucja** (GCN - *Graph Convolutional Network*), która została zaproponowana przez Kipfa w 2016 roku – [artykuł](https://arxiv.org/pdf/1609.02907.pdf). Praca ma już ponad 11 tysięcy cytowań i wiele obecnych GNNów jest oparta na niej. Zaproponowana sieć zostanie wykorzystana w scenariuszu nadzorowanym.

GCN w każdej zdefiniowanej warstwie oblicza nowe cechy wierzchołków $H^{(l+1)}$ na podstawie cech istniejących $H^{(l)}$. Bazując na wzorze wprowadzonym na laboratorium 4, obliczanie cech wykonywane jest następująco

$$H^{(l+1)} = \hat{D}^{-\frac{1}{2}}\hat{A}\hat{D}^{-\frac{1}{2}}H^{(l)}W^{(l)},$$
gdzie:
- $\hat{A} = A + I$ to macierz sąsiedztwa grafu z dołączonymi pętlami zwrotnymi na każdym wierzchołku (krawędź z danego wierzchołka do samego siebie)
- $\hat{D}$ to macierz stopnii węzłów (macierz diagonalna)
- $\hat{D}^{-\frac{1}{2}}\hat{A}\hat{D}^{-\frac{1}{2}}$ to tzw. symetryczna normalizacja macierzy sąsiedztwa
- $W^{(l)}$ to macierz wyuczalnych parametrów

Elementem kluczowym jest dodanie pętli na każdym wierzchołku, dzięki czemu osiąga się uśrednione cechy zarówno sąsiadów jak i danego wierzchołka. Dodatkowe informacje dostarcza symetryczna normalizacja, która uwzględnia stopień danego wierzchołka oraz stopień sąsiada.

In [ ]:
class GCNModel(nn.Module):
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int,
        out_dim: int
    ):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden_dim)
        self.act1 = nn.ReLU()
        self.conv2 = GCNConv(hidden_dim, out_dim)
        self.act2 = nn.Softmax(dim=1)

    def forward(self, x, edge_index):
        z = self.act1(self.conv1(x, edge_index))
        z = self.act2(self.conv2(z, edge_index))
        return z

Wykorzystany GCN posiada dwie warstwy oddzielone funkcją aktywacji ReLU i warstwą Softmax na końcu. W eksperymentach rozmiar sieci był zmieniany, co zostanie opisane w sekcji **Opis rezultatów**.

# Eksperymenty

Eksperymenty przeprwoadzono osobno dla dwóch modeli oraz dla dwóch rozmiarów wektorów osadzeń wierzchołków - 64 i 128. 

### Zbiór danych

Dane do eksperymentów zostały zapisane jako dataset w formacie `pytorch geometric` - `InMemoryDataset`. Oprócz stworzonych wektorów osadzeń, na danych nie zastosowano modyfikacji oraz transformacji. 

In [7]:
display(Code("src/dataset.py"))

import os
from typing import Callable, List, Optional

import torch
import pandas as pd

from torch_geometric.data import (
    Data,
    InMemoryDataset,
)

from networkx import Graph
from gensim.models import KeyedVectors
from torch_geometric.utils.convert import from_networkx
import networkx as nx
from sklearn.model_selection import train_test_split

class EmailEUCore(InMemoryDataset):
    r"""A modified e-mail communication network of a large European research
    institution, taken from the `"Local Higher-order Graph Clustering"
    <https://www-cs.stanford.edu/~jure/pubs/mappr-kdd17.pdf>`_ paper.
    Nodes indicate members of the institution.
    An edge between a pair of members indicates that they exchanged at least
    one email.
    Node labels indicate membership to one of the 37 departments.

    Args:
        root (string): Root directory where the dataset should be saved.
        transform (callable, optional): A function/transform that takes in an
            :obj:`torch_geometric.data.Data` object and returns a transformed
            version. The data object will be transformed before every access.
            (default: :obj:`None`)
        pre_transform (callable, optional): A function/transform that takes in
            an :obj:`torch_geometric.data.Data` object and returns a
            transformed version. The data object will be transformed before
            being saved to disk. (default: :obj:`None`)
    """

    def __init__(
        self,
        root: str,
        embbeddings_path: str,
        data_path: str,
        graph_path: str,
        split: str = "predefined",
        transform: Optional[Callable] = None,
        pre_transform: Optional[Callable] = None,
        random_state: int = 345,
        num_train_per_class: int = 5
    ):
        self._embbeddings_path = embbeddings_path
        self._graph_path = graph_path
        self._data_path = data_path
        self._random_state = random_state
        
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])
        self.split = split
        
        assert self.split in ['predefined', 'full']

        if split == 'full':
            data = self.get(0)
            data.train_mask.fill_(True)
            data.train_mask[data.val_mask | data.test_mask] = False
            self.data, self.slices = self.collate([data])
   
    @property
    def processed_file_names(self) -> str:
        return 'data.pt'
    
    @property
    def processed_dir(self) -> str:
        return self.root
    
    @property
    def num_classes(self) -> int:
        r"""Returns the number of classes in the dataset."""
        y = self.data.y
        if y is None:
            return 0
        elif y.numel() == y.size(0):
            return torch.unique(y).numel()
        else:
            return self.data.y.size(-1)
    
    def process(self):
        
        labels = pd.read_csv(self._data_path, sep = " ")
        vectors = KeyedVectors.load_word2vec_format(
            self._embbeddings_path,
            binary=False
        )
        
        G = nx.read_gpickle(self._graph_path)
        attribs = list(G.nodes[0].keys())
        
        
        if "department_id" not in attribs:
            raise ValueError("department_id or user_id not found in node attributes")
        
        if "user_id" not in list(labels.columns):
            raise ValueError("user_id not found in train_data")
            
        train_ids, test_ids = train_test_split(
            list(labels["user_id"]),
            test_size=0.5,
            stratify=list(labels["department_id"]), 
            random_state=self._random_state
        )
        
        test_ids, valid_ids = train_test_split(
            test_ids,
            test_size=0.5,
            stratify=[labels[labels["user_id"] == y]["department_id"].iloc[0] for y in test_ids], 
            random_state=self._random_state
        )
        
        
        nx.set

W metodzie `process` dokonano połączenia wszystkich wygenerowanych informacji otrzymując obiekt `Data` o następujących polach:
- `x` - wektor osadzeń przypisany do wierzchołka
- `y` - etykieta zespołu
- `train_mask` - maska wybierająca wierzchołki do zbioru treningowego
- `test_mask` - maska wybierająca wierzchołki do zbioru testowego
- `vak_mask` - maska wybierająca wierzchołki do zbioru walidacyjnego

Podział zbioru został dokonany w stosunku 50:25:25, ze względu na obecność klas o bardzo małym procencie pokrycia.

### Metryki i ewaluacja

Eksperymenty będą analizowane pod kątem następujących metryk:
- Uśredniona miara AUC
- Uśredniona miara F1 
- silhoute score i davies_bouldin score

Dodatkowo zostanie wykonana wizualizacja wektorów osadzeń z użyciem PCA i UMAPa.

Ostateczne wyniki będą porównywane na zbiorze testowym.

# Opis rezultatów

## GCN

In [24]:
df = pd.read_csv(RESULTS_PATH / "gcn_stats.csv")
df.sort_values(by="f1_test", ascending=False).head(5)

,model_name,f1_test,auc_test,silhoute,davies-bouldin
1,Supervised_GCN_64,0.703777,0.923506,0.506771,0.928052
9,Supervised_GCN_64,0.663098,0.929885,0.510670,0.898847
2,Supervised_GCN_64,0.649968,0.928168,0.502619,0.946309
7,Supervised_GCN_64,0.645210,0.885619,0.481542,0.974261
11,Supervised_GCN_128,0.633927,0.926006,0.481468,0.911896


In [25]:
generate_gcn_summary(df)

,f1_test,auc_test,silhoute,davies-bouldin
Supervised_GCN_64,0.628 +- 0.042,0.915 +- 0.018,0.484 +- 0.029,0.955 +- 0.04
Supervised_GCN_128,0.576 +- 0.041,0.918 +- 0.011,0.47 +- 0.036,0.937 +- 0.037


In [22]:
df = pd.read_csv(RESULTS_PATH / "gcn_experiment_hidden_dim.csv")
df.sort_values(by="f1_test", ascending=False).head(5)

,model_name,f1_test,auc_test,silhoute,davies-bouldin
20,GCN_64_hd_256,0.652489,0.902628,0.480045,0.893773
19,GCN_64_hd_128,0.638256,0.918727,0.404092,0.941212
18,GCN_64_hd_128,0.634416,0.939023,0.448258,0.911473
31,GCN_64_hd_512,0.617821,0.923005,0.459401,0.968790
9,GCN_64_hd_64,0.606218,0.901353,0.467757,0.957700


In [23]:
generate_gcn_summary(df)

,f1_test,auc_test,silhoute,davies-bouldin
GCN_64_hd_64,0.555 +- 0.04,0.915 +- 0.014,0.474 +- 0.031,0.951 +- 0.043
GCN_64_hd_128,0.55 +- 0.065,0.918 +- 0.011,0.495 +- 0.049,0.915 +- 0.025
GCN_64_hd_256,0.545 +- 0.056,0.909 +- 0.01,0.491 +- 0.044,0.925 +- 0.04
GCN_64_hd_512,0.563 +- 0.036,0.915 +- 0.011,0.487 +- 0.03,0.893 +- 0.039
